## Statistical Hypothesis Testing

In [2]:
# The Science part of data involves forming and testing hypotheses about our data and the processes that generate it
# Observations of random variables from known distributions, this allwos us to make statements about how likely
# those assumptions are to hold

## Example: Flipping A Coin

In [14]:
# Going to test normal distribution
import math

def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma
# We can use normal_cdf to figure out that its realized value lies within or outside a particular interval
# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's now below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

# Finding the nontail region or the symmetric interval around the mean that accounts for a certain level of likelihood
# Interval centered at the mean containing 60% probability, then we find the cutoffs where the upper and lower tails
# contain 20% of the probability (leaving 60%)

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance = tolerance)
    
    low_z = -10.0                                 # normal_cdf(-10) is (very close to) 0
    hi_z = 10.0                                   # normal_cdf(10) is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2                # consider the midpoint
        mid_p = normal_cdf(mid_z)                 # and the cdf's value there
        if mid_p < p:
            # midpoint is still to low, search above it
            low_z = mid_z
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z = mid_z
        else:
            break
        
    return mid_z

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z wfor which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    # tail_p uses the opposite probability factor than the upper normal bounds
    tail_probability = (1 - probability) / 2 # probability in percentage
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma) 
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print("This is normal distribution with mean 500, st_dev , and n = 1000 flips: ", mu_0, sigma_0) # Distributed approximately normally with mean 500 and st_dev 15.8 with coin flip n = 1000

# H0 rejected if X falls outside the bounds given
print("This is the rejected H0 falling outside of normal bounds 95% probability outside mu_0 and sigma_0: ",normal_two_sided_bounds(0.95, mu_0, sigma_0))
print("Lower bound of 469, upper bound of 531. 5% chance of observing X outside this interval")

# Checking if p is NOT .5 but instead 0.55
# Calculate the power of the test

# 95% bounds based on assumption, p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
# Probability between regular (lo, hi) - irregular (0.55%)
power = 1 - type_2_probability
# This shows the coin slighly biased towards heads
print("Type 2 Probability: ", type_2_probability, "Power: ", power)
# Significance Test to find the cutoff below which 95% of the probability lies
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print("This is the upper bound limit: ", hi)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("This is the power: ", power)
# More powerful test, since it no longer rejects H0 when X is below 469 and rejects between 526 and 531

This is normal distribution with mean 500, st_dev , and n = 1000 flips:  500.0 15.811388300841896
This is the rejected H0 falling outside of normal bounds 95% probability outside mu_0 and sigma_0:  (469.01026640487555, 530.9897335951244)
Lower bound of 469, upper bound of 531. 5% chance of observing X outside this interval
Type 2 Probability:  0.11345199870463285 Power:  0.8865480012953671
This is the upper bound limit:  526.0073585242053
This is the power:  0.9363794803307173


## p-values

In [25]:
# Instead of choosing bounds baed on some probability cutoff, we compute the probability that we would see a value
# at least as extreme as the one we actually observed.

# For our two sided test of whether the coin is fair
import random

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)
print("This is 530 heads: ", two_sided_p_value(529.5, mu_0, sigma_0))
print("That would be %.2f " % two_sided_p_value(529.5, mu_0, sigma_0))
# 529.5 is a estimate of the probability of seeing 530 than normal_probability_between(530, 531, mu_0, sigma_0)
# Convincing yourself
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0             # count # of heads
                   for _ in range(1000))                          # in 1000 flips
    if num_heads >= 530 or num_heads <= 470:                      # and count how often
        extreme_value_count += 1                                  # the # is 'extreme'
    
print("The extreme value count is : ", extreme_value_count / 100000)

# The p value is greater than our 5% significance, we don't reject the null. 
# 532 heads, the p-value is this
print("This is 532 heads: ", two_sided_p_value(531.5, mu_0, sigma_0))
# This is smaller than 5% significance, which means we reject the null
# Just a different way of approaching the stats.

# Make sure data is roughly normally distributed beforeusing normal_probability above to compute p-values

This is 530 heads:  0.06207721579598857
That would be 0.06 
The extreme value count is :  0.06135
This is 532 heads:  0.046345287837786575


## Confidence Intervals

In [28]:
# 3rd approach is to construct a confidence interval around the observed value of the parameter
# math.sqrt(p * (1 - p) / 1000)
# Don't know p, so we use our estimate
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print("This is sigma with p_hat: ", sigma)
# The following interval contains the true parameter p
normal_two_sided_bounds(0.95, mu, sigma)
print("This is the confidence interval: ", normal_two_sided_bounds(0.95, mu, sigma))

This is sigma with p_hat:  0.015791611697353755
This is the confidence interval:  (0.4940490278129096, 0.5559509721870904)


## P-hacking

In [30]:
# Procedure that erroneously rejects the null hypothesis only 5% of the time will - by definition - 5% of the time
# erroneously reject the null hypothesis

def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]
def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip]) #
    return num_heads < 469 or num_heads > 531
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])
print("This is the number of rejections: ", num_rejections)
# Determine your hypothesis before looking at your data, clean data with hypothesis in mind
# p values are not substitutes for common sense
# Alternative approach is Bayesian Inference

This is the number of rejections:  46


## Example: Running an A/B Test

In [35]:
# Here is why you would use statistical inference
# 990 / 1000 visitors click the Ad-A while 10 / 1000 click the Ad-B
# This is a pretty good difference but most statistics won't be this way.

# N(A) people see Ad A and n(A) click on it
# p(A) is the probability that someone clicks ad A

def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma
# This math only really works out if you know the standard deviations
# For large datasets it's close enough that it doesn't make much of a difference

# Testing the null hypothesis that p(A) and p(B) are the same (0) using the statistic
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)
# This should be a standard normal
z = a_b_test_statistic(1000, 200, 1000, 180)
print("This is 200 clicks out of 1000 and 180 clicks out of 1000: ", z)

# The probability of seeing such a large difference if the means were actually equal
print("Two sided p value of z: ", two_sided_p_value(z))
# 0.254 Large enough that you can't conclude there's much of a difference
z = a_b_test_statistic(1000, 200, 1000, 150)
two_sided_p_value(z)
print("New z: ", z, "New two sided p value: ", two_sided_p_value(z))
# 0.003 probability you'd see such a large difference if the ads were equally effective

This is 200 clicks out of 1000 and 180 clicks out of 1000:  -1.1403464899034472
Two sided p value of z:  0.254141976542236
New z:  -2.948839123097944 New two sided p value:  0.003189699706216853


## Bayesian Inference

In [ ]:
# Procedures before have involved making probability statements about our tests and hypothesis.
# An alternative approach to inference involves trating the unknown parameters themselves as random vars.
# Rather than making judgements about the tests, make probability judgements about the parameters themselves

# Use a prior from the beta distribution which puts all its probabilities between 0 and 1
# math.gamma = (1 - n)!
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)
# Distribution is centers its weight at alpha / (alpha + beta)
# The larger alpha and beta are, the "higher" the distribution is
